In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('/Users/danielepica/Documents/GitHub/recommender_system_llm/src/books_recommendation.csv')
df = df[:100]

In [3]:
df.head()

,books_info
0,title:The Church of Christ: A Biblical Ecclesi...
1,title:No place to cry: The hurt and healing of...
2,"title:Night World: Daughters Of Darkness, desc..."
3,"title:The Rabbi's Cat, description:Gaining the..."
4,"title:From Potter's Field, description:The six..."


In [4]:
#!pip install chromadb langchain
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
import os

embeddings = OllamaEmbeddings(model='mxbai-embed-large', show_progress=True)
llm = Ollama(model='llama3')

#create a new Chroma vector store (index)
persist_directory = 'db_persist'
#Check if the directory already exists
if os.path.exists(persist_directory):
    #Remove the directory and its contents
    import shutil
    shutil.rmtree(persist_directory)

recommend_db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

# Crea embeddings riga per riga e salva in Chroma
for idx, row in df.iterrows():
    # Estrarre l'informazione combinata
    document_text = row['books_info']
   
    # Aggiungere il documento alla base di dati vettoriale Chroma
    recommend_db.add_texts(texts=[document_text], metadatas=[{'user_id': idx}])

# Persistenza dei dati
recommend_db.persist() #NON è PIù NECESSARIO


AttributeError: 'str' object has no attribute 'page_content'

In [12]:
persist_directory = 'db_persist'
embeddings = OllamaEmbeddings(model='mxbai-embed-large', show_progress=True)
recommend_db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)


In [8]:
#1 Metodo
docs = recommend_db.similarity_search(query4, k=5)
docs

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.53it/s]


[Document(metadata={'user_id': 5}, page_content="title:Economics in one lesson, description:With over a million copies sold, Economics in One Lesson is an essential guide to the basics of economic theory. A fundamental influence on modern libertarianism, Hazlitt defends capitalism and the free market from economic myths that persist to this day. Considered among the leading economic thinkers of the “Austrian School,” which includes Carl Menger, Ludwig von Mises, Friedrich (F.A.) Hayek, and others, Henry Hazlitt (1894-1993), was a libertarian philosopher, an economist, and a journalist. He was the founding vice-president of the Foundation for Economic Education and an early editor of The Freeman magazine, an influential libertarian publication. Hazlitt wrote Economics in One Lesson, his seminal work, in 1946. Concise and instructive, it is also deceptively prescient and far-reaching in its efforts to dissemble economic fallacies that are so prevalent they have almost become a new orthod

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

#2 Metodo
qa = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff", retriever = recommend_db.as_retriever(), return_source_documents=True, verbose=True)

In [ ]:
query = "I'm looking for an drama book. What could you suggest to me?"
result = qa({"query": query})
result['result']

In [6]:
type(recommend_db)

langchain_community.vectorstores.chroma.Chroma

<h1> Content based <h1>

Devo dividere tutti i libri già letti dall'utente selezionato, capire il genere e consigliarne altri simili.

In [13]:
query = "page_content=title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships'], review/score:5.0,title:The Happiest Baby on the Block: The New Way to Calm Crying and Help Your Newborn Baby Sleep Longer, authors:['Harvey Karp'], categories:['FAMILY & RELATIONSHIPS'], review/score:5.0"
query2= "title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships']"
query3 = "title:The Church of Christ: A Biblical Ecclesiology for Today, description:In The Church of Christ: A Biblical Ecclesiology for Today, respected biblical scholar Everett Ferguson presents a genuine biblical theology of the church. By systematically examining the New Testament's teaching on the existence, meaning, and purpose of the church, providing responsible coverage of the traditional topics in ecclesiology, and carefully grounding ecclesiology in the person and work of Christ, Ferguson unveils a comprehensive model of the church that is both biblically centered and relevant to a world on the verge of the twenty-first century., authors:['Everett Ferguson'], categories:['Religion'], ratingsCount:5.0"

query4= '''['[(\'Dream Brother: The Lives and Music of Jeff and Tim Buckley\', "When Jeff Buckley drowned at the age of thirty in 1997, he not only left behind a legacy of brilliant music -- he brought back haunting memories of his father, \'60s troubadour Tim Buckley, a gifted musician who barely knew his son and who himself died at twenty-eight. Both father and son made...", "[\'David Browne\']", "[\'Music\']", \'3.0\')]']'''



In [14]:
#1 Metodo
docs = recommend_db.similarity_search(query4, k=5)
docs
#Non ha senso questo risultato.

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


[Document(metadata={'user_id': 74}, page_content="title:A Rip in Heaven: A Memoir of Murder And Its Aftermath, description:The acclaimed author of American Dirt reveals the devastating effects of a shocking tragedy in this landmark true crime book—the first ever to look intimately at the experiences of both the victims and their families. A Rip in Heaven is Jeanine Cummins’ story of a night in April, 1991, when her two cousins Julie and Robin Kerry, and her brother, Tom, were assaulted on the Old Chain of Rocks Bridge, which spans the Mississippi River just outside of St. Louis. When, after a harrowing ordeal, Tom managed to escape the attackers and flag down help, he thought the nightmare would soon be over. He couldn’t have been more wrong. Tom, his sister Jeanine, and their entire family were just at the beginning of a horrific odyssey through the aftermath of a violent crime, a world of shocking betrayal, endless heartbreak, and utter disillusionment. It was a trial by fire from wh

In [10]:
def find_similar_book(query, vectorstore, top_k=5):
    
    similar_docs = vectorstore.similarity_search_with_score(query, k=5)

    return similar_docs

In [6]:
find_similar_book(query, recommend_db)

NameError: name 'query' is not defined